In [33]:
import numpy as np
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from scipy.sparse import hstack
from scipy.special import logit, expit

class_names = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']

train = pd.read_csv('kaggle-toxic/input/train.csv').fillna(' ').head(10000)
test = pd.read_csv('kaggle-toxic/input/test.csv').fillna(' ').head(10000)

train_text = train['comment_text']
test_text = test['comment_text']
all_text = pd.concat([train_text, test_text])

word_vectorizer = TfidfVectorizer(
   sublinear_tf=True,
   strip_accents='unicode',
   analyzer='word',
   token_pattern=r'\w{1,}',
   ngram_range=(1, 2),
   max_features=10000)
word_vectorizer.fit(all_text)
train_features = word_vectorizer.transform(train_text)
test_features = word_vectorizer.transform(test_text)

# char_vectorizer = TfidfVectorizer(
#     sublinear_tf=True,
#     strip_accents='unicode',
#     analyzer='char',
#     ngram_range=(1, 5),
#     max_features=10000)
# char_vectorizer.fit(all_text)
# train_features = char_vectorizer.transform(train_text)
# test_features = char_vectorizer.transform(test_text)

#train_features = hstack([train_char_features, train_word_features])
#test_features = hstack([test_char_features, test_word_features])


losses = []
predictions = {'id': test['id']}
for class_name in class_names:
    train_target = train[class_name]
    classifier = LogisticRegression(solver='liblinear')

    cv_loss = np.mean(cross_val_score(classifier, train_features, train_target, cv=3, scoring='roc_auc'))
    losses.append(cv_loss)
    print('CV score for class {} is {}'.format(class_name, cv_loss))

    classifier.fit(train_features, train_target)
    predictions[class_name] = classifier.predict_proba(test_features)[:, 1]

print('Total CV score is {}'.format(np.mean(losses)))

submission = pd.DataFrame.from_dict(predictions)
submission.to_csv('kaggle-toxic/input/submission.csv', index=False)

CV score for class toxic is 0.9392792856401654
CV score for class severe_toxic is 0.9767524378099774
CV score for class obscene is 0.9610238465451154
CV score for class threat is 0.9740947415890252
CV score for class insult is 0.9520062387519506
CV score for class identity_hate is 0.9351764978551375
Total CV score is 0.9563888413652286
